<a href="https://colab.research.google.com/github/gmarchiello/Candidate-Data-Workflow-Automation/blob/main/colab/Candidate-Data-Workflow-Automation_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Candidate Data Workflow Automation
This Colab notebook demonstrates a **mockup version** of a tool I originally developed at *Instituto Cervantes* to streamline candidate data updates.

**Overview of the workflow:**
- **Input data:** downloaded automatically from GitHub (Excel templates, candidate information).  
- **Missing data handling:** fields with missing values are safely filled with `"UNKNOWN"`.  
- **Output:**
  - Filled PDFs are saved in **timestamped folders** with safe filenames.  
  - All PDFs are **zipped automatically** and offered for download directly in Colab.  
  - **Logging** in the notebook shows:  
    - ✅ Success  
    - ⚠️ Missing text values  
    - ❗ Missing checkboxes  
  - Batch messages for review (or email preparation) are displayed in **chunks**, summarizing candidates’ changes and highlighting missing data.  
- **Automation:** repetitive tasks like filling PDFs, naming files, and preparing batch summaries are handled automatically, while still allowing manual interventions where needed.


In [ ]:
# --- INSTALL REQUIRED LIBRARIES ---
!pip install pandas pdfrw pytz openpyxl --quiet

import os, re, sys, requests, shutil
import pandas as pd
from datetime import datetime
from pytz import timezone
from pathlib import Path
from pdfrw import PdfReader, PdfWriter, PdfName, PdfObject
from google.colab import files

# --- GITHUB FILE RAW URLs ---
EXCEL_URL = "https://github.com/gmarchiello/pdf_form_filler/raw/main/input/change_request_data/candidates_change_request_tool.xlsx"
PDF_URL = "https://github.com/gmarchiello/pdf_form_filler/raw/main/input/templates/data_form_editable.pdf"

# --- LOCAL TEMPORARY PATHS IN COLAB ---
EXCEL_PATH = Path("candidates_change_request_tool.xlsx")
PDF_PATH = Path("/content/data_form_editable.pdf")
OUTPUT_DIR = Path("/content/output")  # temporary folder for filled PDFs
OUTPUT_DIR.mkdir(exist_ok=True)

# --- DOWNLOAD EXCEL FILE ---
print("⬇️ Downloading Excel file...")
r = requests.get(EXCEL_URL)
with open(EXCEL_PATH, "wb") as f:
    f.write(r.content)

# --- DOWNLOAD PDF FILE ---
print("⬇️ Downloading PDF template...")
r = requests.get(PDF_URL)
with open(PDF_PATH, "wb") as f:
    f.write(r.content)

print("✅ Files downloaded from GitHub")

# --- CONFIGURATION ---
CONFIG = {
    "director": "John Smith",
    "exam_center_city": "Naples",
    "exam_center_country": "Italy",
    "institute_city": "Naples",
    "location": "Naples",
    "chunk_size": 10
}

CHECKBOX_MAP = {
    "chk_gender": "Gender_chk",
    "chk_name": "Name_chk",
    "chk_surname": "Surname_chk",
    "chk_date_of_birth": "Date_of_birth_chk",
    "chk_place_of_birth": "Place_of_birth_chk",
    "chk_country_of_birth": "Country_of_birth_chk",
    "chk_email": "Email_chk",
}

TEXT_MAP = {
    "txt_director": None,
    "txt_exam_center_city": None,
    "txt_exam_center_country": None,
    "txt_institute_city": None,
    "txt_client_code": "Client_code",
    "txt_exam_code": "Exam_code",
    "txt_gender": "Gender",
    "txt_name": "Name",
    "txt_surname": "Surname",
    "txt_country_of_birth": "Country_of_birth",
    "txt_date_of_birth": "Date_of_birth",
    "txt_place_of_birth": "Place_of_birth",
    "txt_email": "Email",
    "txt_location": None,
    "txt_today_date": None,
}

# --- UTILS ---
italy_tz = timezone("Europe/Rome")

def safe_get(value, for_pdf_field=True, placeholder="UNKNOWN"):
    if pd.isna(value) or str(value).strip() in ["", "nan", "NaT"]:
        return "" if for_pdf_field else placeholder
    if isinstance(value, float) and value.is_integer():
        value = int(value)
    return str(value).strip()

def make_output_folder(base_dir):
    base_dir = Path(base_dir)
    timestamp = datetime.now(italy_tz).strftime("%Y%m%d_%H%M")
    output_dir = base_dir / f"fulfilled_forms_{timestamp}"
    output_dir.mkdir(parents=True, exist_ok=True)
    return output_dir

def clean_filename(name, surname, suffix_list):
    suffix_strs = [str(s) for s in suffix_list]
    safe_name = re.sub(
        r"[^a-zA-Z0-9]",
        "_",
        f"{surname}_{name}_change_request_" + "_".join(suffix_strs),
    )
    return safe_name

def get_checked_fields(row, checkbox_map):
    changed = []
    for pdf_checkbox, excel_col in checkbox_map.items():
        if str(row.get(excel_col, "")).strip().upper() == "ON":
            friendly_name = excel_col.replace("_chk", "")
            changed.append(friendly_name)
    return changed

# --- PDF FILLER ---
def fill_pdf(input_pdf_path, output_pdf_path, text_values, checkboxes_to_check):
    pdf = PdfReader(str(input_pdf_path))
    pdf.Root.AcroForm.update({PdfName("NeedAppearances"): PdfObject("true")})

    for page in pdf.pages:
        annotations = page.Annots
        if not annotations:
            continue
        for annot in annotations:
            if annot.Subtype == PdfName.Widget and annot.T:
                key = annot.T.to_unicode().strip()
                if key in text_values:
                    annot.V = text_values[key]
                    annot.AP = None
                elif key in checkboxes_to_check:
                    annot.V = PdfName("Yes")
                    annot.AS = PdfName("Yes")
    PdfWriter().write(str(output_pdf_path), pdf)

# --- TIMEZONE SETUP ---
italy_tz = timezone("Europe/Rome")

# --- LOAD AND VALIDATE FILES ---
try:
    missing_files = []
    if not EXCEL_PATH.exists():
        missing_files.append(str(EXCEL_PATH))
    if not PDF_PATH.exists():
        missing_files.append(str(PDF_PATH))
    if missing_files:
        file_word = "file is" if len(missing_files) == 1 else "files are"
        raise FileNotFoundError(
            f"The following required {file_word} missing:\n - " +
            "\n - ".join(str(f) for f in missing_files)
        )
    print("✅ All required files are present.")

    df = pd.read_excel(str(EXCEL_PATH), sheet_name="PYTHON")
    df.columns = df.columns.str.strip()
    if df.columns.isnull().any() or (df.columns == "").any():
        raise ValueError("Excel contains empty or invalid column headers.")

    print(f"✅ Excel loaded successfully: {EXCEL_PATH}")
    print(f"✅ Column headers normalized and valid: {list(df.columns)}")

    pdf_template = PdfReader(str(PDF_PATH))
    if not pdf_template.Root.AcroForm or not getattr(pdf_template.Root.AcroForm, "Fields", None):
        raise ValueError("PDF template does not contain form fields.")
    print(f"✅ PDF template loaded successfully: {PDF_PATH}")

    pdf_fields = set()
    for field in pdf_template.Root.AcroForm.Fields:
        if field.T:
            pdf_fields.add(field.T.to_unicode().strip())

    missing_text = set(TEXT_MAP.keys()) - pdf_fields
    missing_checkboxes = set(CHECKBOX_MAP.keys()) - pdf_fields
    if missing_text or missing_checkboxes:
        raise ValueError(
            f"PDF template is missing required fields.\n"
            f"Missing text fields: {missing_text}\n"
            f"Missing checkboxes: {missing_checkboxes}"
        )
    else:
        print("✅ PDF contains all required fields.")

    excel_cols = set(df.columns)
    required_text_cols = set(c for c in TEXT_MAP.values() if c is not None)
    required_checkbox_cols = set(CHECKBOX_MAP.values())
    missing_text_cols = required_text_cols - excel_cols
    missing_checkbox_cols = required_checkbox_cols - excel_cols
    if missing_text_cols or missing_checkbox_cols:
        raise ValueError(
            f"Excel is missing required columns.\n"
            f"Missing text columns: {missing_text_cols}\n"
            f"Missing checkbox columns: {missing_checkbox_cols}"
        )
    else:
        print("✅ Excel contains all required columns.")

except FileNotFoundError as e:
    print(f"❌ File not found. Please check your file paths.\nError details: {e}")
    sys.exit(1)
except ValueError as e:
    print(f"❌ Configuration error. Please check your files.\nError details: {e}")
    sys.exit(1)
except Exception as e:
    print(f"❌ An unexpected error occurred.\nError details: {e}")
    sys.exit(1)

# --- OUTPUT FOLDER ---
OUTPUT_FOLDER = make_output_folder(str(OUTPUT_DIR))

# --- PROCESS EXCEL ROWS AND GENERATE FILLED PDFS ---
email_data = []

for idx, row in df.iterrows():
    checked_fields = get_checked_fields(row, CHECKBOX_MAP)
    missing_checkbox = not bool(checked_fields)

    missing_fields = [
        excel_col for excel_col in TEXT_MAP.values()
        if excel_col is not None and not safe_get(row.get(excel_col))
    ]

    suffix_list = []
    if checked_fields:
        suffix_list.append("_".join(checked_fields).replace(" ", "_"))
    if missing_fields:
        suffix_list.append("MISSING_" + "_".join(missing_fields))
    if missing_checkbox:
        suffix_list.append("MISSING_CHECKBOX")

    name = safe_get(row.get("Name"), for_pdf_field=False)
    surname = safe_get(row.get("Surname"), for_pdf_field=False)
    safe_name = clean_filename(name, surname, suffix_list)
    output_pdf_path = OUTPUT_FOLDER / f"{safe_name}.pdf"

    text_values = {}
    for pdf_field, excel_col in TEXT_MAP.items():
        if excel_col:
            value = row.get(excel_col)
            if pdf_field == "txt_date_of_birth":
                birth_date = pd.to_datetime(value, errors="coerce")
                text_values[pdf_field] = birth_date.strftime("%d/%m/%Y") if pd.notnull(birth_date) else ""
            else:
                text_values[pdf_field] = safe_get(value)
        else:
            if pdf_field in ["txt_director", "txt_exam_center_city", "txt_exam_center_country",
                             "txt_institute_city", "txt_location"]:
                text_values[pdf_field] = CONFIG[pdf_field.replace("txt_", "")]
            elif pdf_field == "txt_today_date":
                text_values[pdf_field] = datetime.now(italy_tz).strftime("%d %B %Y")

    checkboxes_to_check = []
    for pdf_field, excel_col in CHECKBOX_MAP.items():
        if str(row.get(excel_col, "")).strip().upper() == "ON":
            checkboxes_to_check.append(pdf_field)

    try:
        fill_pdf(str(PDF_PATH), str(output_pdf_path), text_values, checkboxes_to_check)
    except Exception as e:
        print(f"❌ Error processing row {idx}: {e}")
        continue

    if missing_fields and missing_checkbox:
        print(f"⚠️ Row {idx} missing text value: {', '.join(missing_fields)}")
        print(f"❗ Row {idx} has no checkboxes selected")
        print(f"🟡 Row {idx} processed: {output_pdf_path.name}")
    elif missing_fields:
        print(f"⚠️ Row {idx} missing text value: {', '.join(missing_fields)}")
        print(f"🟡 Row {idx} processed: {os.path.basename(output_pdf_path)}")
    elif missing_checkbox:
        print(f"❗ Row {idx} has no checkboxes selected")
        print(f"🔴 Row {idx} processed: {os.path.basename(output_pdf_path)}")
    else:
        print(f"✔️ Row {idx} processed: {os.path.basename(output_pdf_path)}")

    changed_text = ", ".join(checked_fields) if checked_fields else "No changes"
    body_line = f"{surname} {name} ({changed_text})"
    if missing_fields:
        body_line += " ⚠️ MISSING " + ", ".join([mf.replace("_", " ") for mf in missing_fields])
    if missing_checkbox:
        body_line += " ❗ MISSING CHECKBOX"
    email_data.append((surname, name, body_line))

# --- GENERATE EMAIL MESSAGES ---
email_data_with_changes = [
    entry for entry in email_data
    if "No changes" not in entry[2] or "❗ MISSING CHECKBOX" in entry[2] or "⚠️ MISSING" in entry[2]
]

email_data_with_changes.sort()
chunk_size = CONFIG["chunk_size"]
chunks = [email_data_with_changes[i:i + chunk_size]
          for i in range(0, len(email_data_with_changes), chunk_size)]

for i, chunk in enumerate(chunks):
    subject_surname_name = [f"{entry[1]} {entry[0]}" for entry in chunk]
    email_subject = f"Change Request: {', '.join(subject_surname_name)}"
    email_body_lines = [f"- {entry[2]}" for entry in chunk]
    email_body = "Good morning,\nI kindly ask you to update the data of the following candidates:\n\n" + "\n".join(email_body_lines)
    print(f"\n--- Message {i + 1} ---")
    print(email_subject)
    print("\n" + email_body)
    print("\n-------------------\n")

# --- ZIP ALL GENERATED PDFS ---
zip_filename = str(OUTPUT_FOLDER) + ".zip"
shutil.make_archive(str(OUTPUT_FOLDER), 'zip', str(OUTPUT_FOLDER))
print(f"✅ All filled PDFs zipped as: {zip_filename}")

# --- AUTO DOWNLOAD ZIP FILE ---
files.download(zip_filename)
print("⬇️ Download started for the filled forms zip")

# --- OPTIONAL CLEANUP ---
if EXCEL_PATH.exists():
    EXCEL_PATH.unlink()
if PDF_PATH.exists():
    PDF_PATH.unlink()
if OUTPUT_FOLDER.exists():
    shutil.rmtree(OUTPUT_FOLDER)
print("🗑️ Temporary files removed")


⬇️ Downloading Excel file...
⬇️ Downloading PDF template...
✅ Files downloaded from GitHub
✅ All required files are present.
✅ Excel loaded successfully: candidates_change_request_tool.xlsx
✅ Column headers normalized and valid: ['Client_code', 'Exam_code', 'Name', 'Surname', 'Gender', 'Date_of_birth', 'Place_of_birth', 'Country_of_birth', 'Email', 'Name_chk', 'Surname_chk', 'Gender_chk', 'Date_of_birth_chk', 'Place_of_birth_chk', 'Country_of_birth_chk', 'Email_chk']
✅ PDF template loaded successfully: /content/data_form_editable.pdf
✅ PDF contains all required fields.
✅ Excel contains all required columns.
✔️ Row 0 processed: Fields_John_change_request_Name.pdf
⚠️ Row 1 missing text value: Name
🟡 Row 1 processed: Rivas_UNKNOWN_change_request_Surname_MISSING_Name.pdf
⚠️ Row 2 missing text value: Surname
🟡 Row 2 processed: UNKNOWN_John_change_request_Gender_MISSING_Surname.pdf
✔️ Row 3 processed: Palmer_Marissa_change_request_Date_of_birth.pdf
✔️ Row 4 processed: Morgan_Anthony_change_r

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

⬇️ Download started for the filled forms zip
🗑️ Temporary files removed
